In [ ]:
import numpy as np
import pickle

In [ ]:

folder = "results/20190427_16h10/"

file_prefix = "enreg"

In [32]:
nb_tx = 21
nb_rx = 1
tx_offset = 0
offset = 0
file_re = [[]]
file_im = [[]]
for j in range(tx_offset,nb_tx+tx_offset):
    file_re.append([])
    file_im.append([])
    for i in range(offset, nb_rx+offset):
        file_re[j-tx_offset].append('{0}re_{1:02d}_{2}.bin'.format(folder, i,j))
        file_im[j-tx_offset].append('{0}im_{1:02d}_{2}.bin'.format(folder, i,j))


In [ ]:
packet_size = 600

cut_proportion = 1 #If we don't want to use the whole dataset
print ("Nb de Tx : ", nb_tx)

In [ ]:
for rx in range(nb_rx):
    valeur_re = np.array([])
    valeur_im = np.array([])
    label = None 
    for tx in range(nb_tx):

        tmp_re = np.fromfile(file_re[tx][rx], dtype=np.float32, count=-1)
        tmp_im = np.fromfile(file_im[tx][rx], dtype=np.float32, count=-1)
        nb_p = (tmp_im.shape[0]//packet_size)
        cut_pos = int(nb_p * cut_proportion)
        
        valeur_re = np.append(valeur_re, tmp_re[:packet_size*(cut_pos)]) #Ensure case where files have partially written packets at the end      
        valeur_im = np.append(valeur_im, tmp_im[:packet_size*(cut_pos)])
        
        if label is None:
            label = np.stack(((tx)*np.ones(cut_pos), (rx)*np.ones(cut_pos)), axis = 1)
        else:
            label = np.append(label, np.stack(((tx)*np.ones(cut_pos), (rx)*np.ones(cut_pos)), axis = 1) , axis=0)

            
    nb_packets = valeur_re.shape[0] //packet_size

    
    if (rx == 0):
        vecteur_re = valeur_re[:packet_size*nb_packets].reshape(nb_packets, packet_size)
        vecteur_im = valeur_im[:packet_size*nb_packets].reshape(nb_packets, packet_size)
        labels = label

    else:
        vecteur_re = np.concatenate((vecteur_re, valeur_re[:packet_size*nb_packets].reshape(nb_packets, packet_size)), axis=0)
        vecteur_im = np.concatenate((vecteur_im, valeur_im[:packet_size*nb_packets].reshape(nb_packets, packet_size)), axis=0)
        labels = np.concatenate((labels, label), axis=0)

    
    valeur_re = valeur_re[:packet_size*nb_packets].reshape(nb_packets, packet_size)
    valeur_im = valeur_im[:packet_size*nb_packets].reshape(nb_packets, packet_size)


In [ ]:
pickle_prefix = "pickle_rx"

file_real = open(folder + pickle_prefix + "_real",'wb')
pickle.dump(vecteur_re, file_real, protocol=4)
file_real.close()

file_imag = open(folder + pickle_prefix + "_imag",'wb')
pickle.dump(vecteur_im, file_imag, protocol=4)
file_imag.close()

file_labels = open(folder + pickle_prefix + "_labels",'wb')
pickle.dump(labels, file_labels, protocol=4)
file_labels.close()
